# 공간 분석
> 공간 분석이란 지리공간정보를 데이터로 변환하여 특정 위치나 공간의 특징을 시각적으로 분석해내는 기법을 의미합니다.

## 목차
- import Base code
    - polygon / 폴리곤
    - heatmap / 히트맵
    - marker(point) / 마커(포인트)
    - bubble / 버블
- 함수 시연
    - polygon / 폴리곤
    - heatmap / 히트맵
    - marker(point) / 마커(포인트)
    - bubble / 버블

## import Base code

In [1]:
! pip install geom==1.1

You should consider upgrading via the '/Users/jheok/Desktop/mobigen/Template-Jupyter-Sample/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import json
import os
import pandas as pd
import shapely.geometry

from Geom import search

def geojson2centroid(df):
    result = []
    for row in df.values.tolist():
        juso_code = search.juso_to_code(sido=row[1], sgg=row[2], emd=row[3])
        geojson = search.code_to_geom(juso_code=str(juso_code))
        result.append([
            row[0],
            shapely.geometry.shape(json.loads(geojson)).centroid.x,
            shapely.geometry.shape(json.loads(geojson)).centroid.y,
            row[4]]
        )
    return result

file_path = os.path.join(os.getcwd(), "sample_space.xlsx")

address_df = pd.read_excel(file_path, sheet_name="주소")
latlon_df = pd.read_excel(file_path, sheet_name="위경도")

## polygon / 폴리곤
> 폴리곤이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 지도 위에 특정 포인트를 연결하여 생성한 다각형을 의미합니다.
지도 위에서 특정 포인트를 연결하는 것은 선분으로 나타납니다. 특정 범주에 속하는 인위적인 영역을 묶음 지어 표현하고자 할 때 주로 사용됩니다.

- 구성
    -  polygon 실행함수

In [3]:
# polygon 실행함수

def polygon(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = []
        for row in df.values.tolist():
            juso_code = search.juso_to_code(sido=row[1], sgg=row[2], emd=row[3])
            geojson = search.code_to_geom(juso_code=str(juso_code))
            result.append([row[0], geojson, row[4]])

        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "GEOJSON", "COUNT"]
        )
    else:  # 위경도
        result = []
        for row in df.values.tolist():
            result.append([row[0], f'{{"type":"Point","coordinates":({row[1]}, {row[2]})}}', row[3]])

        df = pd.DataFrame(
            data=[],
            columns=["LABEL", "GEOJSON", "COUNT"]
        )
    return df

## heatmap / 히트맵
> 히트맵이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, Heat(열)와 Map(지도)의 합성어를 뜻합니다.

- 구성
    -  heatmap 실행함수

In [4]:
# heatmap 실행함수

def heatmap(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

## marker(point) / 마커(포인트)
> 마커란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 정해진 특정 포인트를 시각적으로 표현해내는 시각화 방법을 의미합니다.

- 구성
    -  marker(point) 실행함수

In [5]:
# marker(point) 실행함수

def point(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

## bubble / 버블
> 버블이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 위도 및 경도로 기록된 지도 상의 위치에 특정 컬러로 칠해진 원형 도형을 통해 해당 위치에서의 측정값을 시각적으로 표현합니다.

- 구성
    -  bubble 실행함수

In [6]:
# bubble 실행함수

def bubble(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

# 함수 시연
## polygon / 폴리곤

In [7]:
address_polygon_df = polygon(address_df, "주소")
latlon_polygon_df = polygon(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

  LABEL                                            GEOJSON  COUNT
0     A  {"type":"MultiPolygon","coordinates":[[[[35.61...   1498
1     B  {"type":"MultiPolygon","coordinates":[[[[34.48...   4411
2     C  {"type":"MultiPolygon","coordinates":[[[[36.30...   8801
3     D  {"type":"MultiPolygon","coordinates":[[[[35.37...   9762
4     E  {"type":"MultiPolygon","coordinates":[[[[37.72...   8171
5     F  {"type":"MultiPolygon","coordinates":[[[[37.66...  10915
6     G  {"type":"MultiPolygon","coordinates":[[[[36.58...   2849
7     H  {"type":"MultiPolygon","coordinates":[[[[36.73...   7385
8     I  {"type":"MultiPolygon","coordinates":[[[[36.48...   9852
9     J  {"type":"MultiPolygon","coordinates":[[[[35.19...  21579
Empty DataFrame
Columns: [LABEL, GEOJSON, COUNT]
Index: []


## heatmap / 히트맵

In [8]:
address_polygon_df = heatmap(address_df, "주소")
latlon_polygon_df = heatmap(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

  LABEL        LAT         LON  COUNT
0     A  35.565194  127.927992   1498
1     B  34.498927  127.321177   4411
2     C  36.302331  127.376627   8801
3     D  35.379461  128.740500   9762
4     E  37.702589  126.795662   8171
5     F  37.717334  126.963483  10915
6     G  36.582830  127.980244   2849
7     H  36.677921  126.626897   7385
8     I  36.470151  127.136587   9852
9     J  35.174341  129.147882  21579
    LABEL         LAT         LON  가중치(숫자)
0  인천국제공항  337.462500  126.439167    14928
1  김포국제공항   37.558056  126.790556    11568
2  제주국제공항   33.511111  126.492778    13661
3  김해국제공항   35.179444  128.938056     4626


## marker(point) / 마커(포인트)

In [9]:
address_polygon_df = point(address_df, "주소")
latlon_polygon_df = point(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

  LABEL        LAT         LON  COUNT
0     A  35.565194  127.927992   1498
1     B  34.498927  127.321177   4411
2     C  36.302331  127.376627   8801
3     D  35.379461  128.740500   9762
4     E  37.702589  126.795662   8171
5     F  37.717334  126.963483  10915
6     G  36.582830  127.980244   2849
7     H  36.677921  126.626897   7385
8     I  36.470151  127.136587   9852
9     J  35.174341  129.147882  21579
    LABEL         LAT         LON  가중치(숫자)
0  인천국제공항  337.462500  126.439167    14928
1  김포국제공항   37.558056  126.790556    11568
2  제주국제공항   33.511111  126.492778    13661
3  김해국제공항   35.179444  128.938056     4626


## bubble / 버블

In [10]:
address_polygon_df = bubble(address_df, "주소")
latlon_polygon_df = bubble(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

  LABEL        LAT         LON  COUNT
0     A  35.565194  127.927992   1498
1     B  34.498927  127.321177   4411
2     C  36.302331  127.376627   8801
3     D  35.379461  128.740500   9762
4     E  37.702589  126.795662   8171
5     F  37.717334  126.963483  10915
6     G  36.582830  127.980244   2849
7     H  36.677921  126.626897   7385
8     I  36.470151  127.136587   9852
9     J  35.174341  129.147882  21579
    LABEL         LAT         LON  가중치(숫자)
0  인천국제공항  337.462500  126.439167    14928
1  김포국제공항   37.558056  126.790556    11568
2  제주국제공항   33.511111  126.492778    13661
3  김해국제공항   35.179444  128.938056     4626
